doc 是 WPS-2003 及以前的版本
docx 是 WSP-2007 及以后的版本

In [43]:
from docx import Document
import re
import json

In [15]:
file_path = r"/Users/lianke/Desktop/local_test/2018.docx"
world_file = Document(file_path)
paragraphs = world_file.paragraphs 


In [ ]:
def docx_to_dic(docx_path):
    world_file = Document(docx_path) 

    # 1. 循环获取表格数据
    tabel_list = []
    for table in world_file.tables:
        dic = {}
        for row in table.rows:
            dic[row.cells[0].text] = row.cells[1].text
        tabel_list.append(dic)


    # 2. 循环获取段落数据
    system_prog = re.compile("^[\u4e00-\u9fa5]{1,10}、")
    equipment_prog = re.compile("^（[\u4e00-\u9fa5\u767e\u5343\u96f6]{1,10}）")

    dic = {}
    i=0
    for paragraph in world_file.paragraphs:
        text = paragraph.text
        if text=="目 录":continue
        if text != "":
            print(dic)
            if system_prog.match(text):
                current_system = text
                dic.update({current_system:{}})
                print(1)
            elif equipment_prog.match(text):
                current_equipment = text
                dic[current_system].update({current_equipment:{}})
                print(2)
            else:
                try:
                    dic[current_system][current_equipment].update({text:{}})
                    print(3)
                # 系统下只有一个设备，文档会省略掉设备，直接写缺陷
                except KeyError:
                    dic[current_system].update({current_system:{}})
                    dic[current_system][current_system].update({text:{}})
            print(text)
            i+=1
        
    # 3. 将表格数据和段落数据融合
    print(dic)
    i = 0
    for system,v_system in dic.items():
        for equipment,v_equipment in v_system.items():
            for defect in v_equipment.keys():
                dic[system][equipment][defect] = tabel_list[i]
                i += 1

    # with open("/Users/lianke/Desktop/local_test/2018.json","w") as f:
    #     json.dump(dic,f,ensure_ascii=False)

    return dic

In [ ]:
from docx.document import Document
from docx.oxml.table import CT_Tbl
from docx.oxml.text.paragraph import CT_P
from docx.table import _Cell, Table
from docx.text.paragraph import Paragraph
import docx

# 函数迭代器
def iter_block_items(parent):
    if isinstance(parent, Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            yield Paragraph(child, parent)
        elif isinstance(child, CT_Tbl):
            yield Table(child, parent)


tabel_style = ["Normal Table","Table Grid"]
doc = docx.Document('/Users/lianke/Desktop/local_test/2018.docx')

# 每一步循环执行函数的下一步，返回一个block，block可能是段落也可能是表格
for block in iter_block_items(doc):
    
    if block.style.name == tabel_style:
        print(block.style.name)
        dic = {}
        for row in block.rows:
            dic[row.cells[0].text] = row.cells[1].text
        print(dic)
    elif block.text == "":
        pass
    elif block.style.name == 'Heading 1':
        print(block.style.name)
        print(block.text)
    elif block.style.name == 'Heading 2':
        print(block.style.name)
        print(block.text)
    elif block.style.name == 'Heading 3':
        print(block.style.name)
        print(block.text)
    else:
        print(block.style.name)
        print(block.text)
